# Dealing with Outliers

## Options for dealing with them:

1. Fix the data: Look at the data and correct it. Can be costly or impossible. 
2. Censor: Delete observations that are outliers
3. Winsorize: Change the value so that it is closer to the rest of the distribution
    - Example: Any value above the 99th percentile for a variable is changed to equal the 99th percentile
    - This is a common and cheap ad-hoc correction that downplays the weight of the outlier in your analysis because the values are reduced, without tossing out the observation altogether
    - Tough question that depends on the data/application: What is the "right" amount of the distribution to winsorize?

## Finding outliers

How can we find outliers?
 
1. Plot your data: scatterplot, hexplot, box plot, density, etc. 
1. Compute z-scores and `.describe()` the data

Plotting is essential in EDA and data cleaning, as we've covered. I'm going to suggest the second route though as a quick and systematic way to identify which variables you should look into more.

Let's download our firm data (`ccm`).


In [1]:
# copied from 3.3.4.1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# these three are used to download the file
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

url = 'https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/data/CCM_cleaned_for_class.zip?raw=true'

#firms = pd.read_stata(url)   
# <-- that code would work, but GH said it was too big and
# forced me to zip it, so here is the work around to download it:

with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        ccm = pd.read_stata(stata)				



And load a utility function from the [community codebook](https://github.com/LeDataSciFi/ledatascifi-2021/blob/main/community_codebook/outlier_report.py).

In [2]:
def outlier_report(df,vars_to_examine=None,color='red',thres=4,
                   return_df=False,no_print=False):
    '''
    Parameters
    ----------
    df : DATAFRAME
        Input dataframe
    vars_to_examine : LIST, optional
        List of variables to examine from dataframe. The default is df.columns.
    color : STRING, optional
        Color for cell highlighting. The default is 'red'.
    thres : int, optional
        Highlight cells where z score is above thres. The default is 4.
    return_df : Boolean, optional
        If true, will return the df obj (without styling) for further use. 
        The default is False.
    no_print : Boolean, optional
        If true, will not print. 
        The default is False.
    
    Displays (if no_print=False)
    -------
    Table with distribution of z-scores of variables of interest. 
    
    Returns (if return_df=True)
    -------
    Table with distribution of z-scores of variables of interest (without styling).     
    '''
        
    def highlight_extreme(s):
        '''
        Highlight extreme values in a series.
        '''
        is_extreme = abs(s) > thres
        return ['background-color: '+color if v else '' for v in is_extreme]
    
    if vars_to_examine==None:
        vars_to_examine=df.columns
    
    _tab = (
            # compute z scores
            ((df[vars_to_examine] - df[vars_to_examine].mean())/df[vars_to_examine].std())
            # output dist of z   
            .describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]).T
            # add a new column = highest of min and max column
            .assign(max_z_abs = lambda x: x[['min','max']].abs().max(axis=1))
            # now sort on it
            .sort_values('max_z_abs',ascending = False)
    )
    
    if no_print == False:
        display(_tab
             .style.format('{:,.2f}')
                   .format({"count": '{:,.0f}'})           
                   .apply(highlight_extreme, 
                          subset=['mean', 'std', 'min', '1%', '5%', '25%', '50%', '75%', '95%','99%', 'max', 'max_z_abs'])
        ) 
    
    if return_df == True:
        return _tab



Now we can pick the variables we want to check and use our utility function:

In [3]:
vars_to_check = ['l_a', 'l_sale', 'prof_a', 'mb', 'ppe_a', 'capx_a', 'xrd_a', 
                 'cash_a', 'div_d', 'td_a', 'td_mv', 'dltt_a', 'dv_a', 	'invopps_FG09',
                 'sales_g', 'short_debt', 'long_debt_dum', 'atr', 'smalltaxlosscarry', 
                 'largetaxlosscarry', 'tnic3hhi', 'tnic3tsimm', 'prodmktfluid', 
                 'delaycon', 'equitydelaycon', 'debtdelaycon', 'privdelaycon', 'l_emp',
                 'l_ppent', 'l_laborratio']

outlier_report(ccm,vars_to_check,thres=4)

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max,max_z_abs
dv_a,"206,679",0.00,1.00,-0.12,-0.08,-0.08,-0.08,-0.08,-0.01,0.19,0.70,356.75,356.75
sales_g,"196,652",-0.00,1.00,-4.75,-0.03,-0.02,-0.02,-0.01,-0.01,0.01,0.07,281.98,281.98
mb,"219,269",0.00,1.00,-0.20,-0.15,-0.13,-0.10,-0.08,-0.01,0.30,0.96,272.32,272.32
invopps_FG09,"193,956",0.00,1.00,-0.18,-0.15,-0.13,-0.10,-0.07,-0.00,0.29,0.91,261.27,261.27
dltt_a,"222,002",0.00,1.00,-0.56,-0.56,-0.56,-0.53,-0.19,0.32,1.16,2.01,234.93,234.93
td_a,"221,468",0.00,1.00,-0.78,-0.65,-0.65,-0.52,-0.13,0.33,1.10,1.85,199.83,199.83
prof_a,"217,167",-0.00,1.00,-187.91,-0.96,-0.34,-0.03,0.04,0.10,0.20,0.32,171.16,187.91
xrd_a,"223,001",-0.00,1.00,-2.60,-0.25,-0.25,-0.25,-0.25,-0.09,0.97,3.06,112.51,112.51
capx_a,"206,399",0.00,1.00,-23.30,-0.71,-0.71,-0.54,-0.27,0.18,1.62,3.73,95.10,95.10
short_debt,"194,560",0.00,1.00,-15.08,-0.98,-0.98,-0.82,-0.38,0.62,2.04,2.04,67.28,67.28


And you can extract a list of problematic variables from this function too:

In [4]:
vars_with_big_outliers = list(outlier_report(ccm,vars_to_check,thres=4,return_df=True,no_print=True)
                              .query('max_z_abs > 5').index)

## Winsorizing

In this example:
1. If a value is below the 1st percentile, change it to the 1st percentile
1. If a value is above the 99th percentile, change it to the 99th percentile

Options:
1. Winsorize once over whole dataset
1. Winsorize over subgroups (e.g., winsorize by year)
    - Useful when the distribution changes over time
    - Suppose the distribution shifts right from one year to the next. If you winsorize both years at once, you'll chop off the lower values in year one and the upper values in year two. Perhaps it makes more sense to winsorize each year separately.

### Code:

```python
from scipy.stats.mstats import winsorize

# option 1a:
for v in [some_list_of_vars]:
    df[v] = winsorize(df[v],limits=[.01,.99])
    
# option 1b: same as 1a, but one line
df[some_list_of_vars]= df[some_list_of_vars].apply(lambda x: winsorize(x,limits=[.01,.99]))

# option 2: winsorize by group (e.g. year)
df[some_list_of_vars] = df.groupby(some_group)[some_list_of_vars].transform(lambda x: winsorize(x,limits=[.01,.99]))
```

### An example

In [5]:
from scipy.stats.mstats import winsorize

practice_df = ccm.copy() # don't do this in your code,
                         # just run the remaining lines:

# winsorizse one var at a time 
vars_to_win = vars_to_check # decide which, for this example - all the ones we checked
for v in vars_to_win:
    practice_df[v] = winsorize(practice_df[v],limits=[.01,.99])


Let's verify it worked:

In [6]:
(practice_df
 .describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]).T
 # add a new column = highest of min and max column
 .assign(abs_maxmin = lambda x: x[['min','max']].abs().max(axis=1))
 # now sort on it
 .sort_values('abs_maxmin',ascending = False)
 [['min','1%',"99%",'max']] # only need these to see winsorizing in action
 [10:15] # just print a few vars
 .style.format('{:,.2f}')
)    


,min,1%,99%,max
l_sale,-1.64,-1.64,-1.64,-1.64
prof_a,-1.05,-1.05,-1.05,-1.05
tnic3tsimm,1.01,1.01,1.01,1.01
sales_g,-0.73,-0.73,-0.73,-0.73
mb,0.56,0.56,0.56,0.56
